# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Traffic Light Recognition Classifier Combined MobileNet with FabLeNet



---
## Step 0: Load The Data

In [116]:
import sys
import csv
import numpy as np
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
import cv2

%matplotlib inline

GRAPH_MOBILENET="frozen_inference_graph.pb"
inference_map_text=["Red", "Yellow", "Green", "none"]
inference_map_code=['TrafficLight.RED', 'TrafficLight.YELLOW', 'TrafficLight.GREEN', 'TrafficLight.UNKNOWN']

fablenet_map_text=["Red", "Yellow", "Green"]

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def load_graph(graph_file):
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return graph

def to_image_coords (boxes, height, width):
    box_coords=np.zeros_like(boxes)
    box_coords[:, 0] = boxes[:, 0] *  height
    box_coords[:, 1] = boxes[:, 1] * width
    box_coords[:, 2] = boxes[:, 2] * height
    box_coords[:, 3] = boxes[:, 3] * width

    return box_coords

def draw_boxes(cv_image, box_coords, classes):
        for box, cls in zip(box_coords, classes):
            color = (255,255,255)
            if cls == 10:
                color = (0,0,255)
            cv2.rectangle(cv_image, (box[1],box[0]), (box[3], box[2]), color=color, thickness=2)

class TLClassifier(object):
    def __init__(self):
        detection_graph = load_graph(GRAPH_MOBILENET)

        # The input placeholder for the image.
        self.image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')

        # The classification of the object (integer id).
        self.detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

        self.tf_session = tf.Session(graph=detection_graph)
        

    def mobilenet_preprocess(self, cv_image):
        x = cv_image/128.
        x -= 0.5
        return x
    
    def perform_detect_ssd(self, cv_image):
        cv_image = cv2.resize(cv_image, (300, 300))
        preprocessed_input = self.mobilenet_preprocess(cv_image)
        (boxes, scores, classes) = self.tf_session.run([self.detection_boxes, self.detection_scores, self.detection_classes],
                                            feed_dict={self.image_tensor: (cv_image, )})

        # Remove unnecessary dimensions
        boxes = np.squeeze(boxes)
        scores = np.squeeze(scores)
        classes = np.squeeze(classes)

        n = len(classes)
        min_score = 0.25
        idxs=[]
        tl_idxs=[]
        for i in range(n):
            if scores[i] >= min_score:
                idxs.append(i)
                if classes[i]==10:
                    tl_idxs.append(i)

        filtered_boxes = boxes[idxs, ...]
        filtered_scores = scores[idxs, ...]
        filtered_classes = classes[idxs, ...]

        #print(filtered_scores, filtered_classes)
        height, width, _ = cv_image.shape
        box_coords = to_image_coords(filtered_boxes, height, width)

        draw_boxes(cv_image, box_coords, filtered_classes)

        return cv_image, boxes[tl_idxs, ...]


#mobile_graph = load_graph(GRAPH_MOBILENET)
#tf_session = tf.Session(graph=mobile_graph)
#op_list = mobile_graph.get_operations()
#print(len(op_list))
#with open('mobilenet_tensors.txt', 'w') as f:
#    for l in op_list:
#        f.write(l.name+'\n')
def resize32(s_img):
    x=s_img.shape[1]
    y=s_img.shape[0]
    if (x>32 or y>32):
        if (x>y):
            s_img=cv2.resize(s_img, (32, y*32//x))
        else:
            s_img=cv2.resize(s_img, (x*32//y, 32))
    l_img = np.zeros((32,32,3), dtype=np.uint8)
    x_offset = (32-s_img.shape[1])//2
    y_offset = (32-s_img.shape[0])//2
    l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1]] = s_img
    return l_img

def modify_images(X_data):
    arr=X_data.copy()
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
    for idx in range(len(arr)):
        arr[idx] = cv2.cvtColor(arr[idx], cv2.COLOR_RGB2LAB)
        l_clahe = clahe.apply(arr[idx,:,:,0])
        arr[idx,:,:,0] = l_clahe
        m = np.max(arr[idx,:,:,0])
        if (m>1):
            arr[idx,:,:,0]=np.uint8(np.float32(arr[idx,:,:,0])*255/m)
        arr[idx] = cv2.cvtColor(arr[idx], cv2.COLOR_LAB2RGB)
    return (np.float32(arr[:,:,:,:])-128)/128

FabLeNet_graph = load_graph("model/fablenet.pb")
FabLeNet = tf.Session(graph=FabLeNet_graph)
#tf.train.Saver().restore(FabLeNet, "./lenet")
#top5_operation = tf.nn.top_k(tf.nn.softmax(logits), 5)

#op_list = FabLeNet_graph.get_operations()
#print(len(op_list))
#with open('fablenetnet_tensors.txt', 'w') as f:
#    for l in op_list:
#        f.write(l.name+'\n')

fablenet_input = FabLeNet_graph.get_tensor_by_name('input:0')
fablenet_dropkeep = FabLeNet_graph.get_tensor_by_name('Placeholder_1:0')
fablenet_result = FabLeNet_graph.get_tensor_by_name('result:0')
top3_operation = tf.nn.top_k(tf.nn.softmax(fablenet_result), 3)
    

In [118]:

classifier = TLClassifier()

def annotate_image(test1):
    #test1 = cv2.imread('lisa_yellow2.jpg')
    #test1 = cv2.imread('test_green.png')
    #test1 = cv2.imread('just_0386.png')
    test1rgb = cv2.cvtColor(test1, cv2.COLOR_BGR2RGB)
    #test1 = cv2.imread('test_red.png')
    #plt.figure(figsize=(12, 8))
    #plt.imshow(test1)
    #plt.show()
    res1, filtered_boxes=classifier.perform_detect_ssd(test1rgb)
    #print(filtered_boxes, test1.shape)
    box_coords = to_image_coords(filtered_boxes, test1rgb.shape[0], test1rgb.shape[1])
    
    line_idx=0
    for box in box_coords:
        #print(box)
        box=box.astype(int)
        if box[2]>box[0] and box[3]>box[1]:
            crop=test1rgb[box[0]:box[2],box[1]:box[3]]
            #plt.figure(figsize=(3, 3))
            #plt.imshow(crop)
            #plt.show()
            res=resize32(crop)
            ##res=cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
            #plt.figure(figsize=(3, 3))
            #plt.imshow(res)
            #plt.show()
            top3 = FabLeNet.run(top3_operation, feed_dict={fablenet_input: modify_images(np.array([res])), fablenet_dropkeep: 1.0})
            for idx in range(len(top3.values)):
                for n in range(len(top3.values[0])):
                    #print(fablenet_map_text[top3.indices[idx, n]], "{:.4f}".format(top3.values[idx, n]))
                    #draw.text((0, line_idx*10), "%s {:.4f}".format( fablenet_map_text[top3.indices[idx, n]], top3.values[idx, n]), (255, 255, 255), font=font)
                    text="{} {:.4f}".format( fablenet_map_text[top3.indices[idx, n]], top3.values[idx, n])
                    cv2.putText(res1,text, (10,line_idx*15+20), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255,255,255),1)
                    line_idx+=1
    return res1

import os
dirname='img_just_416/'

for i in os.listdir(dirname):
    print(i)
    test1 = cv2.imread(dirname+i)
    res1=annotate_image(test1)
    print('out_'+dirname+i)
    cv2.imwrite('out_'+dirname+i, cv2.cvtColor(res1, cv2.COLOR_RGB2BGR))

print("ready.")
#plt.figure(figsize=(12, 8))
#plt.imshow(res1)
#plt.show()

just_0000.png
out_img_just_416/just_0000.png
just_0001.png
out_img_just_416/just_0001.png
just_0002.png
out_img_just_416/just_0002.png
just_0003.png
out_img_just_416/just_0003.png
just_0004.png
out_img_just_416/just_0004.png
just_0005.png
out_img_just_416/just_0005.png
just_0006.png
out_img_just_416/just_0006.png
just_0007.png
out_img_just_416/just_0007.png
just_0008.png
out_img_just_416/just_0008.png
just_0009.png
out_img_just_416/just_0009.png
just_0010.png
out_img_just_416/just_0010.png
just_0011.png
out_img_just_416/just_0011.png
just_0012.png
out_img_just_416/just_0012.png
just_0013.png
out_img_just_416/just_0013.png
just_0014.png
out_img_just_416/just_0014.png
just_0015.png
out_img_just_416/just_0015.png
just_0016.png
out_img_just_416/just_0016.png
just_0017.png
out_img_just_416/just_0017.png
just_0018.png
out_img_just_416/just_0018.png
just_0019.png
out_img_just_416/just_0019.png
just_0020.png
out_img_just_416/just_0020.png
just_0021.png
out_img_just_416/just_0021.png
just_0022.

out_img_just_416/just_0182.png
just_0183.png
out_img_just_416/just_0183.png
just_0184.png
out_img_just_416/just_0184.png
just_0185.png
out_img_just_416/just_0185.png
just_0186.png
out_img_just_416/just_0186.png
just_0187.png
out_img_just_416/just_0187.png
just_0188.png
out_img_just_416/just_0188.png
just_0189.png
out_img_just_416/just_0189.png
just_0190.png
out_img_just_416/just_0190.png
just_0191.png
out_img_just_416/just_0191.png
just_0192.png
out_img_just_416/just_0192.png
just_0193.png
out_img_just_416/just_0193.png
just_0194.png
out_img_just_416/just_0194.png
just_0195.png
out_img_just_416/just_0195.png
just_0196.png
out_img_just_416/just_0196.png
just_0197.png
out_img_just_416/just_0197.png
just_0198.png
out_img_just_416/just_0198.png
just_0199.png
out_img_just_416/just_0199.png
just_0200.png
out_img_just_416/just_0200.png
just_0201.png
out_img_just_416/just_0201.png
just_0202.png
out_img_just_416/just_0202.png
just_0203.png
out_img_just_416/just_0203.png
just_0204.png
out_img_ju

out_img_just_416/just_0364.png
just_0365.png
out_img_just_416/just_0365.png
just_0366.png
out_img_just_416/just_0366.png
just_0367.png
out_img_just_416/just_0367.png
just_0368.png
out_img_just_416/just_0368.png
just_0369.png
out_img_just_416/just_0369.png
just_0370.png
out_img_just_416/just_0370.png
just_0371.png
out_img_just_416/just_0371.png
just_0372.png
out_img_just_416/just_0372.png
just_0373.png
out_img_just_416/just_0373.png
just_0374.png
out_img_just_416/just_0374.png
just_0375.png
out_img_just_416/just_0375.png
just_0376.png
out_img_just_416/just_0376.png
just_0377.png
out_img_just_416/just_0377.png
just_0378.png
out_img_just_416/just_0378.png
just_0379.png
out_img_just_416/just_0379.png
just_0380.png
out_img_just_416/just_0380.png
just_0381.png
out_img_just_416/just_0381.png
just_0382.png
out_img_just_416/just_0382.png
just_0383.png
out_img_just_416/just_0383.png
just_0384.png
out_img_just_416/just_0384.png
just_0385.png
out_img_just_416/just_0385.png
just_0386.png
out_img_ju

out_img_just_416/just_0548.png
just_0549.png
out_img_just_416/just_0549.png
just_0550.png
out_img_just_416/just_0550.png
just_0551.png
out_img_just_416/just_0551.png
just_0552.png
out_img_just_416/just_0552.png
just_0553.png
out_img_just_416/just_0553.png
just_0554.png
out_img_just_416/just_0554.png
just_0555.png
out_img_just_416/just_0555.png
just_0556.png
out_img_just_416/just_0556.png
just_0557.png
out_img_just_416/just_0557.png
just_0558.png
out_img_just_416/just_0558.png
just_0559.png
out_img_just_416/just_0559.png
just_0560.png
out_img_just_416/just_0560.png
just_0561.png
out_img_just_416/just_0561.png
just_0562.png
out_img_just_416/just_0562.png
just_0563.png
out_img_just_416/just_0563.png
just_0564.png
out_img_just_416/just_0564.png
just_0565.png
out_img_just_416/just_0565.png
just_0566.png
out_img_just_416/just_0566.png
just_0567.png
out_img_just_416/just_0567.png
just_0568.png
out_img_just_416/just_0568.png
just_0569.png
out_img_just_416/just_0569.png
just_0570.png
out_img_ju